# Programmatic Access to EMBL EBI #
### (Exploring Biological Sequences 2019) ###
******





All the resources at EMBL-EBI are freely available and can be explored from https://www.ebi.ac.uk/services. You can access the resources either using a browser interface or programmatically i.e., using web services. A list of EMBL-EBI Web Services APIs for data retrieval resources is given in https://bit.ly/EMBL-EBI-APIs 

Here are some examples to access some of our services programmatically: 

## 1. Retrieve data from DbFetch ##

[DbFetch](https://www.ebi.ac.uk/Tools/dbfetch/) provides an easy way to retrieve entries from various databases at the EMBL-EBI in a consistent manner.

You can retrieve data from DbFetch using our web interface http://www.ebi.ac.uk/Tools/dbfetch/

Required parameters: 
1. Database name
2. Sequence id

Optional Parameters
1. Style (default: html)
2. Format (default format is often the default format for the specified database)



#### Retrieve data from DbFetch using REST URL ###

To retrieve a protein sequence entry (example WAP_MOUSE) from the Uniprot Knowledgebase, we could open a browser window and try the following URL
https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=uniprotkb;id=WAP_MOUSE
 

database is _uniprotkb_ and the sequence id is _WAP_RAT_

https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=uniprotkb;id=WAP_MOUSE;style=raw 


#### Run from command line ####

You can retrieve the data using _curl_ or _wget_ commands

In [ ]:
!curl -X GET "https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=uniprotkb;id=WAP_MOUSE;style=raw"

In [ ]:
!curl -X GET 'https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=uniprotkb;id=WAP_MOUSE;style=raw' -o wap_mouse.txt

In [ ]:
!head -n 30 wap_mouse.txt

In [ ]:
!curl -X GET 'https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=uniprotkb;id=WAP_MOUSE;style=raw;format=fasta' -o wap_mouse.fasta

In [ ]:
!head wap_mouse.fasta

#### Incorporate the calls into a script ####
Suppose you want to bulk download a set of sequence ids in fasta format
Create a file (eg: data.txt) with the list of sequence ids (WAP_RAT,WAP_PIG).

In [ ]:
# Using shell command

!for line in `cat data.txt`; do curl -X GET "https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=uniprotkb;id=$line;style=raw;format=fasta" -o $line.fasta; done

Using file upload
save the file as upload.txt in specified format (eg:uniprotkb:WAP_RAT)and use _CURL_ to _POST_ the request

In [ ]:
!pwd

In [ ]:
!curl -X POST http://www.ebi.ac.uk/Tools/dbfetch/dbfetch -F "listfile=@/mnt/c/Training/upload.txt" -F "format=fasta" -F "style=raw" 

#### Using Web Service Client ####


Python, Perl and Java clients are provided for EBI Tools Web Services from https://github.com/ebi-wp/webservice-clients.

To simplify the process, we can download one of the clients (e.g. DBfetch) and run it in alternative to using the curl command.

Get the raw client from GitHub

In [ ]:
!wget https://raw.githubusercontent.com/ebi-wp/webservice-clients/master/python/dbfetch.py

One can learn more about available parameters and how to use them by typing python <client_name>.py --help

In [ ]:
!python dbfetch.py --help

Retrieving the same sequence in fasta format from Uniprot could be done using the Python client as follows:

In [ ]:
!python dbfetch.py fetchData uniprotkb:WAP_MOUSE fasta

If the above command failed, some dependencies might be missing. See instructions for installing them in https://github.com/ebi-wp/webservice-clients

## 2. Run NCBI Blast+ ##

In addition to data retrieval, EMBL-EBI provides Web Services for popular Bioinformatics Applications such as NCBI BLAST+, Clustal Omega, InterProScan 5, and HMMER. Programmatic access to these services can be explored from https://www.ebi.ac.uk/Tools/webservices. The common API can be browsed from https://www.ebi.ac.uk/Tools/common/tools/help/

Since data needs to be passed to the server for the application to run (i.e. some input sequence data, in this case), the request uses the POST HTTP verb.
In this example, we can run NCBI BLAST+ using Swissprot database (uniprotkb_swissprot) and using a UniProt sequence accession number as the input.

### 1.Run NCBI Blast+ using REST URL


In [ ]:
!curl -X POST --header 'Content-Type: application/x-www-form-urlencoded' --header 'Accept: text/plain' -d 'email=test@ebi.ac.uk&program=blastp&stype=protein&sequence=sp:wap_rat&database=uniprotkb_swissprot' 'https://www.ebi.ac.uk/Tools/services/rest/ncbiblast/run'

As you can see, in this case we passed `-X POST` instead of `-X GET`. Additionally, we needed to pass `-d or --data`. The various parameter options were separated with `&` symbol. 
The parameters were:
* `email=test\@ebi.ac.uk`
* `program=blastp`
* `stype=protein` 
* `sequence=sp:wap_rat`
* `database=uniprotkb_swissprot`

For retrieving a BLAST output for the previous job with (example `jobid=ncbiblast-R20190225-110224-0172-5825946-p1m`) we need to use a different endpoint `/result/{jobId}/{resultType}`. In this case we can retrieve the default BLAST output format which has been named `out`.

In [ ]:
!curl -X GET --header 'Accept: text/plain' 'https://www.ebi.ac.uk/Tools/services/rest/ncbiblast/result/ncbiblast-R20191008-111833-0999-94082917-p1m/out' -o blast.txt

In [ ]:
!head -n 50 blast.txt

### 2. Run NCBI Blast+ using Web Service Client

Similarly to what we have done for Dbfetch, we can download the clients and perform various sequence analysis using the available Bioinformatics Applications. In this example we run the same BLAST sequence search as we performed with curl.

In [ ]:
# note: we are getting the raw client from GitHub
!wget https://raw.githubusercontent.com/ebi-wp/webservice-clients/master/python/ncbiblast.py

In [ ]:
# the clients can run the job and return outputs on the same call (synchronously as below, or asynchronously)
!python ncbiblast.py --email test@ebi.ac.uk --program blastp --stype protein --sequence sp:wap_rat --database uniprotkb_swissprot --outformat out --outfile wap_rat

### Useful links

EMBL-EBI services and data resources: https://www.ebi.ac.uk/services  
EMBL-EBI APIs:https://bit.ly/EMBL-EBI-APIs  
EMBL-EBI Web Services General Documentation: https://www.ebi.ac.uk/Tools/webservices  
Web Service Clients for EBI Tools and EBI Search: https://github.com/ebi-wp/webservice-clients  
RESTful API (SWAGGER) User Interface for EBI Tools: https://www.ebi.ac.uk/Tools/common/tools/help/  
RESTful API (SWAGGER) User Interface for EBI Search: https://www.ebi.ac.uk/ebisearch/swagger.ebi  

**Contact us via Help and Support at https://www.ebi.ac.uk/support/webservices**